In [2]:
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt

from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense
from keras.layers import Input
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.metrics import MeanAbsolutePercentageError
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

-    datadate - Date of the Data
-    tic - Ticker Symbol
-    conm - Company Name
-    datacqtr - Year and Quarter of the Data
-    atq - Assets - Total
-    chq - Cash
-    cogsq - Cost of Goods Sold
-    ibq - Income Before Extrodinary Items
-    invtq - Inventories - Total
-    saleq - Sales/Turnover (Net)
-    xrdq - Research and Development Expenses
-    prccq - Price Close - Quarter

In [3]:
data = pd.read_csv("hwdata.csv")

In [4]:
data = data.drop(['Unnamed: 0','Unnamed: 0.1'], axis = 1)
data

,datadate,tic,conm,datacqtr,atq,chq,cogsq,dlttq,ibq,invtq,saleq,xrdq,xsgaq,prccq
0,19620331,BA,BOEING CO,1962Q1,NaN,NaN,NaN,NaN,8.041,NaN,466.8,NaN,NaN,50.1250
1,19620630,BA,BOEING CO,1962Q2,NaN,NaN,NaN,NaN,7.439,NaN,366.9,NaN,NaN,36.7500
2,19620930,BA,BOEING CO,1962Q3,NaN,NaN,NaN,NaN,6.099,NaN,434.7,NaN,NaN,37.9999
3,19621231,BA,BOEING CO,1962Q4,NaN,NaN,NaN,NaN,5.575,NaN,500.1,NaN,NaN,37.1249
4,19630331,BA,BOEING CO,1963Q1,NaN,NaN,NaN,NaN,4.410,NaN,483.8,NaN,NaN,36.2499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5468,20181231,PGR,PROGRESSIVE CORP-OHIO,2018Q4,46575.0,69.5,7666.0,4404.9,264.700,0.0,8028.8,NaN,NaN,60.3300
5469,20190331,PGR,PROGRESSIVE CORP-OHIO,2019Q1,48850.3,158.0,7608.4,4616.8,1078.400,0.0,9293.8,NaN,NaN,72.0900
5470,20190630,PGR,PROGRESSIVE CORP-OHIO,2019Q2,51324.2,91.9,8073.5,4610.2,979.400,0.0,9444.5,NaN,NaN,79.9300
5471,20190930,PGR,PROGRESSIVE CORP-OHIO,2019Q3,54008.1,45.9,8394.6,4610.6,841.700,0.0,9524.6,NaN,NaN,77.2500


In [5]:
hwdatasorted = data.sort_values(by = ['tic', 'datacqtr']).reset_index(drop = True)
hwdatasorted

,datadate,tic,conm,datacqtr,atq,chq,cogsq,dlttq,ibq,invtq,saleq,xrdq,xsgaq,prccq
0,19620331,BA,BOEING CO,1962Q1,NaN,NaN,NaN,NaN,8.041,NaN,466.8,NaN,NaN,50.1250
1,19620630,BA,BOEING CO,1962Q2,NaN,NaN,NaN,NaN,7.439,NaN,366.9,NaN,NaN,36.7500
2,19620930,BA,BOEING CO,1962Q3,NaN,NaN,NaN,NaN,6.099,NaN,434.7,NaN,NaN,37.9999
3,19621231,BA,BOEING CO,1962Q4,NaN,NaN,NaN,NaN,5.575,NaN,500.1,NaN,NaN,37.1249
4,19630331,BA,BOEING CO,1963Q1,NaN,NaN,NaN,NaN,4.410,NaN,483.8,NaN,NaN,36.2499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5468,20181231,XRX,XEROX HOLDINGS CORP,2018Q4,14874.0,1084.0,1371.0,4269.0,137.000,818.0,2533.0,78.0,639.0,19.7600
5469,20190331,XRX,XEROX HOLDINGS CORP,2019Q1,14799.0,723.0,1168.0,4533.0,84.000,859.0,2180.0,77.0,638.0,31.9800
5470,20190630,XRX,XEROX HOLDINGS CORP,2019Q2,14684.0,712.0,1246.0,4216.0,141.000,788.0,2263.0,73.0,605.0,35.4100
5471,20190930,XRX,XEROX HOLDINGS CORP,2019Q3,14660.0,922.0,1179.0,3465.0,157.000,758.0,2179.0,8.4,610.0,29.9100


In [6]:
tics = list(hwdatasorted['tic'].unique())
tics[0:5]

['BA', 'CMA', 'COP', 'DIS', 'DTE']

In [7]:
hwdatasorted['stock_return'] = (hwdatasorted.groupby('tic')['prccq'].apply(pd.Series.pct_change))
hwdatasorted

,datadate,tic,conm,datacqtr,atq,chq,cogsq,dlttq,ibq,invtq,saleq,xrdq,xsgaq,prccq,stock_return
0,19620331,BA,BOEING CO,1962Q1,NaN,NaN,NaN,NaN,8.041,NaN,466.8,NaN,NaN,50.1250,NaN
1,19620630,BA,BOEING CO,1962Q2,NaN,NaN,NaN,NaN,7.439,NaN,366.9,NaN,NaN,36.7500,-0.266833
2,19620930,BA,BOEING CO,1962Q3,NaN,NaN,NaN,NaN,6.099,NaN,434.7,NaN,NaN,37.9999,0.034011
3,19621231,BA,BOEING CO,1962Q4,NaN,NaN,NaN,NaN,5.575,NaN,500.1,NaN,NaN,37.1249,-0.023026
4,19630331,BA,BOEING CO,1963Q1,NaN,NaN,NaN,NaN,4.410,NaN,483.8,NaN,NaN,36.2499,-0.023569
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5468,20181231,XRX,XEROX HOLDINGS CORP,2018Q4,14874.0,1084.0,1371.0,4269.0,137.000,818.0,2533.0,78.0,639.0,19.7600,-0.267606
5469,20190331,XRX,XEROX HOLDINGS CORP,2019Q1,14799.0,723.0,1168.0,4533.0,84.000,859.0,2180.0,77.0,638.0,31.9800,0.618421
5470,20190630,XRX,XEROX HOLDINGS CORP,2019Q2,14684.0,712.0,1246.0,4216.0,141.000,788.0,2263.0,73.0,605.0,35.4100,0.107255
5471,20190930,XRX,XEROX HOLDINGS CORP,2019Q3,14660.0,922.0,1179.0,3465.0,157.000,758.0,2179.0,8.4,610.0,29.9100,-0.155323


In [8]:
cleaned = hwdatasorted[hwdatasorted['stock_return'].notnull()] #Remove the first (null) of each company?
cleaned

,datadate,tic,conm,datacqtr,atq,chq,cogsq,dlttq,ibq,invtq,saleq,xrdq,xsgaq,prccq,stock_return
1,19620630,BA,BOEING CO,1962Q2,NaN,NaN,NaN,NaN,7.439,NaN,366.9,NaN,NaN,36.7500,-0.266833
2,19620930,BA,BOEING CO,1962Q3,NaN,NaN,NaN,NaN,6.099,NaN,434.7,NaN,NaN,37.9999,0.034011
3,19621231,BA,BOEING CO,1962Q4,NaN,NaN,NaN,NaN,5.575,NaN,500.1,NaN,NaN,37.1249,-0.023026
4,19630331,BA,BOEING CO,1963Q1,NaN,NaN,NaN,NaN,4.410,NaN,483.8,NaN,NaN,36.2499,-0.023569
5,19630630,BA,BOEING CO,1963Q2,NaN,NaN,NaN,NaN,4.521,NaN,427.6,NaN,NaN,35.3750,-0.024135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5468,20181231,XRX,XEROX HOLDINGS CORP,2018Q4,14874.0,1084.0,1371.0,4269.0,137.000,818.0,2533.0,78.0,639.0,19.7600,-0.267606
5469,20190331,XRX,XEROX HOLDINGS CORP,2019Q1,14799.0,723.0,1168.0,4533.0,84.000,859.0,2180.0,77.0,638.0,31.9800,0.618421
5470,20190630,XRX,XEROX HOLDINGS CORP,2019Q2,14684.0,712.0,1246.0,4216.0,141.000,788.0,2263.0,73.0,605.0,35.4100,0.107255
5471,20190930,XRX,XEROX HOLDINGS CORP,2019Q3,14660.0,922.0,1179.0,3465.0,157.000,758.0,2179.0,8.4,610.0,29.9100,-0.155323


In [9]:
cleaned['datadate'] = pd.to_datetime(cleaned['datadate'], format='%Y%m%d') 
# three features
cleaned["quarter"] = cleaned['datadate'].dt.quarter
cleaned['ibq'] = cleaned.groupby('tic')['ibq'].transform(lambda v: v.bfill())
cleaned['saleq'] = cleaned.groupby('tic')['saleq'].transform(lambda v: v.bfill())

C:\Users\leech\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\leech\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\leech\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyW

In [10]:
evalresults = pd.DataFrame()
# evalresults[ticsymbol + ' MAPE'] = history.history['mean_absolute_percentage_error']
# evalresults[ticsymbol + ' VAL_MAPE'] = history.history['val_mean_absolute_percentage_error']
# pd.DataFrame(history.history['mean_absolute_percentage_error'], columns = [ticsymbol + ' MAPE'])

In [24]:
individual

,datadate,tic,conm,datacqtr,atq,chq,cogsq,dlttq,ibq,invtq,saleq,xrdq,xsgaq,prccq,stock_return,quarter
5242,1962-06-30,XRX,XEROX HOLDINGS CORP,1962Q2,NaN,NaN,NaN,NaN,3.221,NaN,25.6,NaN,NaN,112.6248,-0.268669,2
5243,1962-09-30,XRX,XEROX HOLDINGS CORP,1962Q3,NaN,NaN,NaN,NaN,3.600,NaN,27.2,NaN,NaN,133.8748,0.188680,3
5244,1962-12-31,XRX,XEROX HOLDINGS CORP,1962Q4,NaN,NaN,NaN,NaN,4.636,NaN,40.9,NaN,NaN,158.3747,0.183006,4
5245,1963-03-31,XRX,XEROX HOLDINGS CORP,1963Q1,NaN,NaN,NaN,NaN,5.010,NaN,36.6,NaN,NaN,160.3748,0.012629,1
5246,1963-06-30,XRX,XEROX HOLDINGS CORP,1963Q2,NaN,NaN,NaN,NaN,5.740,NaN,42.5,NaN,NaN,195.2498,0.217459,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5468,2018-12-31,XRX,XEROX HOLDINGS CORP,2018Q4,14874.0,1084.0,1371.0,4269.0,137.000,818.0,2533.0,78.0,639.0,19.7600,-0.267606,4
5469,2019-03-31,XRX,XEROX HOLDINGS CORP,2019Q1,14799.0,723.0,1168.0,4533.0,84.000,859.0,2180.0,77.0,638.0,31.9800,0.618421,1
5470,2019-06-30,XRX,XEROX HOLDINGS CORP,2019Q2,14684.0,712.0,1246.0,4216.0,141.000,788.0,2263.0,73.0,605.0,35.4100,0.107255,2
5471,2019-09-30,XRX,XEROX HOLDINGS CORP,2019Q3,14660.0,922.0,1179.0,3465.0,157.000,758.0,2179.0,8.4,610.0,29.9100,-0.155323,3


In [21]:
evalresults = pd.DataFrame()
#scaler = MinMaxScaler()
for ticsymbol in tics:
    individual = cleaned[cleaned['tic'] == ticsymbol]
    #standard = pd.DataFrame(scaler.fit_transform(individual[['saleq', 'ibq', 'prccq']]), columns = ['saleq', 'ibq', 'prccq'])
    # split a multivariate sequence into samples
    def split_sequences(sequences, n_steps):
        X, y = list(), list()
        for i in range(len(sequences)):
            # find the end of this pattern
            end_ix = i + n_steps
            # check if we are beyond the dataset
            if end_ix > len(sequences)-1:
                break
            # gather input and output parts of the pattern
            seq_x= sequences[i:end_ix, :]
            X.append(seq_x)
        return array(X)
    # define input sequence
    in_seq1 = array(individual.quarter.tolist())
    #in_seq2 = array(individual.prccq.tolist())
    in_seq2 = array(individual.stock_return.tolist())
    in_seq3 = array(individual.ibq.tolist())
    in_seq4 = array(individual.saleq.tolist())
    # define output sequence, the Response Variable
    #out_seq = array(individual.prccq.tolist())
    out_seq = array(individual.stock_return.tolist())
    

    # convert to [rows, columns] structure
    in_seq1 = in_seq1.reshape((len(in_seq1), 1))
    in_seq2 = in_seq2.reshape((len(in_seq2), 1))
    in_seq3 = in_seq2.reshape((len(in_seq3), 1))
    in_seq4 = in_seq2.reshape((len(in_seq4), 1))

    out_seq = out_seq.reshape((len(out_seq), 1))

    # horizontally stack columns
    dataset = hstack((in_seq1, in_seq2, in_seq3, in_seq4))
    # choose a number of time steps
    n_steps = 8

    # convert into input/output
    X = split_sequences(dataset, n_steps)
    y = out_seq[n_steps:]

    test_size = int(0.2 * len(X))           
    X_train, X_test, y_train, y_test = X[:-test_size], X[-test_size:], y[:-test_size], y[-test_size:]
    # the dataset knows the number of features, e.g. 2
    n_features = X.shape[2]

    # define model
    model = Sequential()
    model.add(Flatten())
    model.add(Dense(64, activation = 'relu', input_shape = (n_steps, n_features)))
    #model.add(Dense(100, activation = 'relu', input_shape = (n_steps, n_features)))
    #model.add(Dense(32, activation = 'relu'))
    #model.add(Dense(50, activation = 'relu'))
    model.add(Dense(1))
    model.compile(optimizer = 'adam', loss = 'mse', metrics = [MeanAbsolutePercentageError()])

    history = model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 100, batch_size = 10)
    
    #evalresults[ticsymbol + ' pred'] = model.predict(X_test).squeeze()
    evalresults[ticsymbol + ' MAPE'] = history.history['mean_absolute_percentage_error']
    evalresults[ticsymbol + ' VAL_MAPE'] = history.history['val_mean_absolute_percentage_error']

#Change to whichever you are predicting    
# prccqresults = evalresults 
# prccqdescription = prccqresults.describe()
# prccqresults.to_csv('FFNN PRCCQ Results.csv')
# prccqdescription.to_csv('FFNN PRCCQ Description.csv')
stockresults = evalresults
stockdescription = stockresults.describe()
stockresults.to_csv('FFNN Stock Return Results.csv')
stockdescription.to_csv('FFNN Stock Return Description.csv')

Train on 179 samples, validate on 44 samples
Epoch 1/100
179/179 [==============================] - 0s 1ms/step - loss: 0.3259 - mean_absolute_percentage_error: 2016.0939 - val_loss: 0.1883 - val_mean_absolute_percentage_error: 6651042.0000
Epoch 2/100
179/179 [==============================] - 0s 220us/step - loss: 0.1050 - mean_absolute_percentage_error: 1568.4932 - val_loss: 0.0383 - val_mean_absolute_percentage_error: 1129203.5000
Epoch 3/100
179/179 [==============================] - 0s 217us/step - loss: 0.0580 - mean_absolute_percentage_error: 526.2728 - val_loss: 0.0347 - val_mean_absolute_percentage_error: 3353536.2500
Epoch 4/100
179/179 [==============================] - 0s 145us/step - loss: 0.0440 - mean_absolute_percentage_error: 263.7104 - val_loss: 0.0177 - val_mean_absolute_percentage_error: 290804.2188
Epoch 5/100
179/179 [==============================] - 0s 157us/step - loss: 0.0411 - mean_absolute_percentage_error: 219.5573 - val_loss: 0.0204 - val_mean_absolute_pe

179/179 [==============================] - 0s 493us/step - loss: 0.0258 - mean_absolute_percentage_error: 315.5326 - val_loss: 0.0331 - val_mean_absolute_percentage_error: 1356471.5000
Epoch 43/100
179/179 [==============================] - 0s 482us/step - loss: 0.0261 - mean_absolute_percentage_error: 386.9289 - val_loss: 0.0239 - val_mean_absolute_percentage_error: 130056.7812
Epoch 44/100
179/179 [==============================] - 0s 527us/step - loss: 0.0241 - mean_absolute_percentage_error: 391.6819 - val_loss: 0.0388 - val_mean_absolute_percentage_error: 2258193.0000
Epoch 45/100
179/179 [==============================] - 0s 475us/step - loss: 0.0267 - mean_absolute_percentage_error: 486.5562 - val_loss: 0.0307 - val_mean_absolute_percentage_error: 1079187.7500
Epoch 46/100
179/179 [==============================] - 0s 207us/step - loss: 0.0245 - mean_absolute_percentage_error: 460.7683 - val_loss: 0.0296 - val_mean_absolute_percentage_error: 1319585.7500
Epoch 47/100
179/179 [==

179/179 [==============================] - 0s 531us/step - loss: 0.0197 - mean_absolute_percentage_error: 335.7581 - val_loss: 0.0290 - val_mean_absolute_percentage_error: 1478302.5000
Epoch 84/100
179/179 [==============================] - 0s 531us/step - loss: 0.0193 - mean_absolute_percentage_error: 340.6131 - val_loss: 0.0340 - val_mean_absolute_percentage_error: 1121884.0000
Epoch 85/100
179/179 [==============================] - 0s 496us/step - loss: 0.0191 - mean_absolute_percentage_error: 314.6220 - val_loss: 0.0454 - val_mean_absolute_percentage_error: 2261806.5000
Epoch 86/100
179/179 [==============================] - 0s 479us/step - loss: 0.0214 - mean_absolute_percentage_error: 479.5969 - val_loss: 0.0320 - val_mean_absolute_percentage_error: 8860.8320
Epoch 87/100
179/179 [==============================] - 0s 513us/step - loss: 0.0208 - mean_absolute_percentage_error: 323.2959 - val_loss: 0.0314 - val_mean_absolute_percentage_error: 1262150.6250
Epoch 88/100
179/179 [====

179/179 [==============================] - 0s 471us/step - loss: 0.0165 - mean_absolute_percentage_error: 776343.6875 - val_loss: 0.0254 - val_mean_absolute_percentage_error: 200.5201
Epoch 25/100
179/179 [==============================] - 0s 486us/step - loss: 0.0164 - mean_absolute_percentage_error: 799369.1250 - val_loss: 0.0234 - val_mean_absolute_percentage_error: 201.9542
Epoch 26/100
179/179 [==============================] - 0s 480us/step - loss: 0.0167 - mean_absolute_percentage_error: 711640.0625 - val_loss: 0.0221 - val_mean_absolute_percentage_error: 212.1445
Epoch 27/100
179/179 [==============================] - 0s 451us/step - loss: 0.0162 - mean_absolute_percentage_error: 858645.1250 - val_loss: 0.0243 - val_mean_absolute_percentage_error: 186.4484
Epoch 28/100
179/179 [==============================] - 0s 463us/step - loss: 0.0164 - mean_absolute_percentage_error: 540913.5000 - val_loss: 0.0247 - val_mean_absolute_percentage_error: 213.5044
Epoch 29/100
179/179 [======

179/179 [==============================] - 0s 488us/step - loss: 0.0146 - mean_absolute_percentage_error: 926260.5000 - val_loss: 0.0243 - val_mean_absolute_percentage_error: 271.0546
Epoch 66/100
179/179 [==============================] - 0s 513us/step - loss: 0.0152 - mean_absolute_percentage_error: 993053.7500 - val_loss: 0.0223 - val_mean_absolute_percentage_error: 289.1795
Epoch 67/100
179/179 [==============================] - 0s 504us/step - loss: 0.0161 - mean_absolute_percentage_error: 603261.1250 - val_loss: 0.0270 - val_mean_absolute_percentage_error: 366.5598
Epoch 68/100
179/179 [==============================] - 0s 391us/step - loss: 0.0178 - mean_absolute_percentage_error: 2010371.0000 - val_loss: 0.0232 - val_mean_absolute_percentage_error: 291.2621
Epoch 69/100
179/179 [==============================] - 0s 302us/step - loss: 0.0147 - mean_absolute_percentage_error: 1007195.2500 - val_loss: 0.0252 - val_mean_absolute_percentage_error: 303.4109
Epoch 70/100
179/179 [====

...

179/179 [==============================] - 0s 373us/step - loss: 0.0140 - mean_absolute_percentage_error: 74142.9922 - val_loss: 0.0097 - val_mean_absolute_percentage_error: 187.0851
Epoch 22/100
179/179 [==============================] - 0s 372us/step - loss: 0.0122 - mean_absolute_percentage_error: 19700.0215 - val_loss: 0.0097 - val_mean_absolute_percentage_error: 217.4345
Epoch 23/100
179/179 [==============================] - 0s 412us/step - loss: 0.0123 - mean_absolute_percentage_error: 89020.8906 - val_loss: 0.0101 - val_mean_absolute_percentage_error: 206.5131
Epoch 24/100
179/179 [==============================] - 0s 432us/step - loss: 0.0127 - mean_absolute_percentage_error: 341572.9688 - val_loss: 0.0117 - val_mean_absolute_percentage_error: 166.9443
Epoch 25/100
179/179 [==============================] - 0s 513us/step - loss: 0.0147 - mean_absolute_percentage_error: 509413.3125 - val_loss: 0.0111 - val_mean_absolute_percentage_error: 232.6640
Epoch 26/100
179/179 [=========

Epoch 63/100
179/179 [==============================] - 0s 150us/step - loss: 0.0135 - mean_absolute_percentage_error: 660936.6875 - val_loss: 0.0111 - val_mean_absolute_percentage_error: 253.0872
Epoch 64/100
179/179 [==============================] - 0s 206us/step - loss: 0.0117 - mean_absolute_percentage_error: 398825.5312 - val_loss: 0.0149 - val_mean_absolute_percentage_error: 229.4242
Epoch 65/100
179/179 [==============================] - 0s 256us/step - loss: 0.0120 - mean_absolute_percentage_error: 147912.3750 - val_loss: 0.0104 - val_mean_absolute_percentage_error: 207.0815
Epoch 66/100
179/179 [==============================] - 0s 255us/step - loss: 0.0114 - mean_absolute_percentage_error: 80849.4766 - val_loss: 0.0121 - val_mean_absolute_percentage_error: 303.3729
Epoch 67/100
179/179 [==============================] - 0s 256us/step - loss: 0.0135 - mean_absolute_percentage_error: 442106.2500 - val_loss: 0.0141 - val_mean_absolute_percentage_error: 204.8394
Epoch 68/100
179

179/179 [==============================] - 0s 234us/step - loss: 0.0424 - mean_absolute_percentage_error: 365.3849 - val_loss: 0.3174 - val_mean_absolute_percentage_error: 211.1490
Epoch 5/100
179/179 [==============================] - 0s 251us/step - loss: 0.0415 - mean_absolute_percentage_error: 235.0883 - val_loss: 0.3092 - val_mean_absolute_percentage_error: 185.3336
Epoch 6/100
179/179 [==============================] - 0s 234us/step - loss: 0.0380 - mean_absolute_percentage_error: 233.0297 - val_loss: 0.2797 - val_mean_absolute_percentage_error: 188.3965
Epoch 7/100
179/179 [==============================] - 0s 257us/step - loss: 0.0384 - mean_absolute_percentage_error: 339.4487 - val_loss: 0.2682 - val_mean_absolute_percentage_error: 171.7387
Epoch 8/100
179/179 [==============================] - 0s 251us/step - loss: 0.0375 - mean_absolute_percentage_error: 271.6670 - val_loss: 0.2874 - val_mean_absolute_percentage_error: 165.3667
Epoch 9/100
179/179 [==========================

179/179 [==============================] - 0s 451us/step - loss: 0.0216 - mean_absolute_percentage_error: 272.2099 - val_loss: 0.3236 - val_mean_absolute_percentage_error: 218.8131
Epoch 47/100
179/179 [==============================] - 0s 465us/step - loss: 0.0204 - mean_absolute_percentage_error: 232.5403 - val_loss: 0.3170 - val_mean_absolute_percentage_error: 228.8554
Epoch 48/100
179/179 [==============================] - 0s 481us/step - loss: 0.0205 - mean_absolute_percentage_error: 231.7887 - val_loss: 0.3159 - val_mean_absolute_percentage_error: 243.4338
Epoch 49/100
179/179 [==============================] - 0s 490us/step - loss: 0.0217 - mean_absolute_percentage_error: 284.0894 - val_loss: 0.3238 - val_mean_absolute_percentage_error: 239.6646
Epoch 50/100
179/179 [==============================] - 0s 440us/step - loss: 0.0219 - mean_absolute_percentage_error: 225.8054 - val_loss: 0.3440 - val_mean_absolute_percentage_error: 221.0876
Epoch 51/100
179/179 [=====================

Epoch 88/100
179/179 [==============================] - 0s 173us/step - loss: 0.0177 - mean_absolute_percentage_error: 323.9816 - val_loss: 0.4111 - val_mean_absolute_percentage_error: 222.0729
Epoch 89/100
179/179 [==============================] - 0s 161us/step - loss: 0.0181 - mean_absolute_percentage_error: 209.7367 - val_loss: 0.4002 - val_mean_absolute_percentage_error: 253.3505
Epoch 90/100
179/179 [==============================] - 0s 171us/step - loss: 0.0168 - mean_absolute_percentage_error: 268.4562 - val_loss: 0.4076 - val_mean_absolute_percentage_error: 252.5573
Epoch 91/100
179/179 [==============================] - 0s 221us/step - loss: 0.0191 - mean_absolute_percentage_error: 318.2046 - val_loss: 0.4011 - val_mean_absolute_percentage_error: 239.4772
Epoch 92/100
179/179 [==============================] - 0s 248us/step - loss: 0.0177 - mean_absolute_percentage_error: 309.2436 - val_loss: 0.4011 - val_mean_absolute_percentage_error: 237.5619
Epoch 93/100
179/179 [========

In [19]:
prccqresults

,BA MAPE,BA VAL_MAPE,CMA MAPE,CMA VAL_MAPE,COP MAPE,COP VAL_MAPE,DIS MAPE,DIS VAL_MAPE,DTE MAPE,DTE VAL_MAPE,...,SLB MAPE,SLB VAL_MAPE,TMO MAPE,TMO VAL_MAPE,VFC MAPE,VFC VAL_MAPE,XOM MAPE,XOM VAL_MAPE,XRX MAPE,XRX VAL_MAPE
0,24.304295,12.806489,31.952250,21.853954,56.576427,48.282658,40.554947,12.926249,61.259579,9.569192,...,28.754961,17.648272,22.698757,15.265685,35.430622,30.828115,23.484877,8.890417,49.670109,37.205898
1,20.599321,12.262460,19.209610,20.057356,36.471416,22.832245,30.529522,18.909840,25.131443,13.834076,...,21.405836,18.359396,21.795532,6.954508,23.774645,25.616800,19.552744,8.567428,39.437111,28.423323
2,20.134436,13.849837,16.875374,18.908173,26.853401,23.606575,24.451372,10.375094,12.571473,16.679686,...,18.891672,13.340121,20.289469,7.933068,20.011353,22.227703,18.779863,9.786106,33.680462,25.231712
3,19.467657,19.237192,15.541778,17.706665,23.886015,19.898787,23.883133,10.254009,13.012476,12.233496,...,18.009668,19.254908,19.988760,10.390550,17.762764,19.394087,16.662214,7.898626,28.140169,22.522745
4,18.620594,12.016427,14.459806,16.785692,22.440546,18.206188,23.759001,14.367950,11.706939,10.793580,...,17.828363,16.161882,19.284704,14.515131,16.046728,18.819895,14.463887,13.216041,25.324301,21.185677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,15.493279,11.561460,10.956336,11.781632,13.278412,13.505222,20.008312,11.230340,7.504560,6.624173,...,14.351382,11.746073,16.357208,13.979739,14.500099,19.964720,9.593558,8.136752,18.889368,35.440655
96,16.783081,12.214209,10.754537,11.705991,12.741123,10.487953,19.406311,10.429820,7.661102,5.406652,...,15.683804,12.922837,16.114273,12.498027,14.123409,18.165619,9.126672,7.093100,18.838360,34.379971
97,15.725412,15.451261,11.163145,11.690810,13.535062,11.424747,20.365847,12.256240,7.407744,9.886655,...,17.595367,11.769955,14.900708,14.158505,13.659099,15.858489,9.165425,6.807059,19.985556,34.349941
98,15.331106,13.425629,10.811503,12.780416,13.674646,10.144004,18.362406,11.431507,8.204249,8.831728,...,17.079727,12.868852,14.923800,12.906437,13.729022,18.573082,10.079634,7.538553,19.205378,37.724430


In [20]:
prccqdescription

,BA MAPE,BA VAL_MAPE,CMA MAPE,CMA VAL_MAPE,COP MAPE,COP VAL_MAPE,DIS MAPE,DIS VAL_MAPE,DTE MAPE,DTE VAL_MAPE,...,SLB MAPE,SLB VAL_MAPE,TMO MAPE,TMO VAL_MAPE,VFC MAPE,VFC VAL_MAPE,XOM MAPE,XOM VAL_MAPE,XRX MAPE,XRX VAL_MAPE
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,16.794519,13.219679,11.985000,12.598897,14.999965,12.795096,20.919505,12.451544,9.168668,8.210564,...,15.773188,13.389736,16.701901,12.868375,15.233342,17.040251,11.174544,8.355994,22.437306,32.094890
std,1.258623,2.055917,2.384283,1.794376,5.323958,4.511331,2.541970,1.743589,5.635603,2.145414,...,1.873671,2.550992,1.407784,3.202329,2.450671,2.363842,2.346023,1.969024,3.912611,7.261360
min,15.111590,10.457763,10.744922,11.437852,12.313501,10.143296,18.362406,10.254009,7.234123,5.261128,...,13.993591,11.256297,14.876917,6.857456,13.363910,14.608184,8.947076,6.641951,18.422976,19.678581
25%,16.047159,11.530826,11.098270,11.680328,13.071098,10.513899,19.733342,11.347426,7.716364,6.568378,...,14.652876,11.722006,15.785788,10.801873,14.294850,15.707783,9.773759,7.063192,20.552763,27.146744
50%,16.409464,12.547908,11.396808,11.875088,13.678389,11.385415,20.239102,12.229956,8.012227,7.781499,...,15.275308,12.153774,16.358961,12.739544,14.853700,16.179818,10.424426,7.829564,21.777832,32.799984
75%,17.261793,14.614475,11.876237,12.703210,14.566181,13.330541,21.338455,12.922542,9.037271,9.709260,...,16.492458,14.059464,17.124672,14.972068,15.462227,17.750903,11.681076,8.692060,22.839317,37.294498
max,24.304295,20.039152,31.952250,21.853954,56.576427,48.282658,40.554947,19.606682,61.259579,16.679686,...,28.754961,25.437750,22.698757,23.299147,35.430622,30.828115,23.484877,17.215446,49.670109,50.145058


In [22]:
stockresults

,BA MAPE,BA VAL_MAPE,CMA MAPE,CMA VAL_MAPE,COP MAPE,COP VAL_MAPE,DIS MAPE,DIS VAL_MAPE,DTE MAPE,DTE VAL_MAPE,...,SLB MAPE,SLB VAL_MAPE,TMO MAPE,TMO VAL_MAPE,VFC MAPE,VFC VAL_MAPE,XOM MAPE,XOM VAL_MAPE,XRX MAPE,XRX VAL_MAPE
0,2016.093872,6.651042e+06,9.831613e+06,1561.678955,6.050369e+06,509.709686,1502.840576,405.213776,9.047052e+06,2057.612305,...,2.594176e+06,292.749420,517.815430,549.093140,1.035803e+07,1369.396851,1.439964e+06,375.956360,2498.886963,568.285095
1,1568.493164,1.129204e+06,1.752454e+06,631.926270,2.292199e+06,747.781616,561.442200,182.273163,5.916605e+06,710.157837,...,1.051722e+06,129.281677,369.157623,262.676514,7.397348e+05,747.163452,5.315598e+05,200.038681,562.584839,283.376587
2,526.272827,3.353536e+06,1.017743e+06,542.640198,1.718926e+06,224.186752,383.501343,132.397812,1.657272e+06,633.369141,...,9.172720e+04,173.295761,234.437119,431.723083,1.828674e+06,618.538330,1.789892e+05,200.322906,507.777222,249.309036
3,263.710388,2.908042e+05,1.730940e+06,399.085266,1.152134e+06,276.020538,349.927704,104.751976,9.580647e+05,245.429794,...,7.733237e+04,164.021301,242.074860,193.211914,4.197007e+05,509.272949,3.453203e+05,273.748444,365.384918,211.149017
4,219.557343,1.036910e+06,1.088036e+06,413.854858,1.138803e+06,333.309937,321.048767,101.240814,5.617606e+05,167.695297,...,8.228379e+04,150.902527,191.864822,229.402435,3.856814e+05,505.551025,2.490313e+05,195.419876,235.088333,185.333633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,254.600342,8.510973e+04,1.952153e+06,385.307251,1.887017e+05,413.934784,204.906708,159.970352,1.074124e+06,425.648315,...,2.138010e+05,195.118851,168.717712,353.289520,1.122977e+06,522.361938,2.777436e+05,253.613632,207.356567,227.205856
96,375.921906,4.821657e+05,5.778015e+05,408.307404,4.870639e+05,433.667358,326.938202,134.917419,7.732892e+05,309.825897,...,6.518083e+05,181.032089,228.626587,225.658203,1.222715e+05,499.813110,2.472878e+05,325.253662,171.684402,256.211792
97,329.554810,1.313571e+06,1.349094e+06,332.160614,4.400876e+05,486.860352,217.744003,124.731697,5.824691e+05,225.318710,...,4.653171e+05,155.244232,171.475525,324.402618,7.985092e+05,514.087463,3.131076e+05,222.673447,292.629852,267.330872
98,278.694916,5.364656e+05,1.481664e+06,398.964966,5.588493e+05,455.049500,213.663239,146.240448,8.192690e+05,299.980164,...,8.639237e+05,177.870941,173.749680,406.411041,1.079606e+06,506.432220,7.968601e+04,227.130020,319.582977,243.461182


In [23]:
stockdescription

,BA MAPE,BA VAL_MAPE,CMA MAPE,CMA VAL_MAPE,COP MAPE,COP VAL_MAPE,DIS MAPE,DIS VAL_MAPE,DTE MAPE,DTE VAL_MAPE,...,SLB MAPE,SLB VAL_MAPE,TMO MAPE,TMO VAL_MAPE,VFC MAPE,VFC VAL_MAPE,XOM MAPE,XOM VAL_MAPE,XRX MAPE,XRX VAL_MAPE
count,100.000000,1.000000e+02,1.000000e+02,100.000000,1.000000e+02,100.000000,100.000000,100.000000,1.000000e+02,100.000000,...,1.000000e+02,100.000000,100.000000,100.000000,1.000000e+02,100.000000,1.000000e+02,100.000000,100.000000,100.000000
mean,388.464875,9.018057e+05,1.138139e+06,301.774114,8.500353e+05,354.934725,271.306945,121.493014,7.102145e+05,277.087429,...,3.620028e+05,176.125291,185.088637,291.431625,5.429171e+05,530.784787,2.541307e+05,242.101947,293.260092,218.272059
std,223.792643,8.720103e+05,9.434827e+05,148.207028,6.414978e+05,120.419198,135.811005,32.265772,1.021241e+06,196.709402,...,3.233908e+05,21.943255,51.225162,114.952815,1.040307e+06,89.803328,1.962164e+05,59.614617,233.299723,51.703785
min,189.743240,3.083742e+03,2.579486e+05,155.573593,1.648052e+05,149.896240,186.222244,94.959694,1.001141e+05,167.695297,...,6.099149e+02,129.281677,113.783241,107.974579,3.577388e+03,482.693359,2.088889e+03,150.278519,160.240479,134.257828
25%,290.452675,2.913139e+05,7.960705e+05,233.501755,5.468213e+05,257.645210,218.313492,107.940239,4.539958e+05,214.581165,...,1.669907e+05,164.460423,155.722309,207.973286,2.202979e+05,506.366798,1.250207e+05,195.374027,228.167004,183.368046
50%,357.791641,8.282214e+05,1.017229e+06,287.812424,7.431948e+05,350.545746,251.138016,116.436035,5.362332e+05,236.939018,...,3.357746e+05,171.185272,173.128265,268.068726,3.677075e+05,521.269165,2.354253e+05,228.125603,253.642120,226.806107
75%,425.826187,1.320441e+06,1.252590e+06,332.800552,9.951869e+05,448.334656,286.995316,126.156298,7.056370e+05,283.824936,...,4.703014e+05,181.336224,203.028366,348.388191,6.632793e+05,530.371155,3.475514e+05,277.856819,301.868980,243.440628
max,2016.093872,6.651042e+06,9.831613e+06,1561.678955,6.050369e+06,747.781616,1502.840576,405.213776,9.047052e+06,2057.612305,...,2.594176e+06,292.749420,517.815430,704.020874,1.035803e+07,1369.396851,1.439964e+06,411.513367,2498.886963,568.285095


In [ ]:
plt.plot(history.history['mean_absolute_percentage_error'])
plt.plot(history.history['val_mean_absolute_percentage_error'])
plt.legend(['train', 'test'], loc='upper left')
plt.xlabel('epoch')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['train', 'test'], loc='upper left')
plt.xlabel('epoch')
plt.show()